In [1]:
from __future__ import division
from __future__ import print_function

import os

import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np

import tensorflow as tf
import matplotlib as plt


tf.enable_eager_execution()

In [2]:
TRAIN_CSV = "../data/train.csv"
EVAL_CSV = "../data/train.csv"
RECORD_DEFAULTS = [[0]] + [[0.] for _ in range(28*28)]

In [3]:
# Data processing cell.
def parse_fn(csv_row):
    row_as_tensor = tf.decode_csv(csv_row, record_defaults=RECORD_DEFAULTS)
    pixels_in_row = row_as_tensor[1:]
    # decode_csv has the batch size as the last dimension
    pixels_in_row = tf.transpose(pixels_in_row)
    with tf.control_dependencies([tf.assert_equal(tf.shape(pixels_in_row)[1:], [28*28])]):
        pixels_in_row = tf.identity(pixels_in_row)
    pixels = tf.reshape(pixels_in_row, shape=[-1, 28, 28, 1])
    label = row_as_tensor[:1]
    label = tf.one_hot(indices=label, depth=10)
    features = {
        "pixels" : pixels
    }
    
    return features, label
    
def get_input_fn(file_pattern, shuffle=False, batch_size=1, num_epochs=1):
    
    def input_fn():
        #files = tf.data.Dataset.list_files(file_pattern)
        dataset = tf.data.TextLineDataset(file_pattern)
        dataset = dataset.skip(1)
        if shuffle:
            dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)
        dataset = dataset.map(parse_fn)
        

        return dataset

    return input_fn


In [4]:
# Inspect data.
def show_array(a, mode='L', fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a, mode=mode).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))


input_fn = get_input_fn(TRAIN_CSV, shuffle=False, batch_size=3)
dataset = input_fn()

for x in dataset.take(1):
    print(x[1])
    for arr in x[0]['pixels']:
        arr2 = tf.squeeze(arr, axis=2)
        show_array(arr2)

tf.Tensor(
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 3, 10), dtype=float32)


In [5]:
# Define model.
def model_fn(features, labels, mode, params):
    layer = features["pixels"]
    
    layer = tf.layers.Conv2D(filters=6, kernel_size=5, strides=1, padding='valid')(layer)
    layer = tf.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid')(layer)
    layer = tf.layers.Conv2D(filters=16, kernel_size=5, strides=1, padding='valid')(layer)
    layer = tf.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid')(layer)
    layer = tf.layers.Dense(units=120, activation=tf.nn.relu)(layer)
    layer = tf.layers.Dense(units=84, activation=tf.nn.relu)(layer)
    layer = tf.layers.Flatten()(layer)
    
    logits = tf.layers.Dense(units=10)(layer)
    labels = tf.squeeze(labels, axis=[0])

    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
    #loss = tf.Print(loss, [loss], message="loss")
    
    # Return the appropriate EstimatorSpec based on the mode.
    probabilities = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(probabilities, axis=-1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {"predicted_labels": predicted_labels, "probabilities": probabilities}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    elif mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy" : tf.metrics.accuracy(labels=tf.argmax(labels, axis=-1), predictions=predicted_labels),
        }
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
    elif mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
#         learning_rate = tf.train.exponential_decay(
#             learning_rate=1e-5, 
#             global_step=global_step,
#             decay_steps=1,
#             decay_rate=1.01,
#         )
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = params['optimizer']
    
        train_op = optimizer.minimize(loss, global_step=global_step)
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
    else:
        assert False, "mode must of type tf.estimator.ModeKeys was {}".format(mode)

    

In [6]:
# Find the right learning rate.
# Train and evaulate.
train_spec = tf.estimator.TrainSpec(input_fn=get_input_fn(TRAIN_CSV, shuffle=True, batch_size=256, num_epochs=1))
eval_spec = tf.estimator.EvalSpec(input_fn=get_input_fn(EVAL_CSV))
estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        params={"optimizer": tf.train.AdamOptimizer(learning_rate=0.0015)},
    )
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12afab410>, '_model_dir': '/var/folders/5h/vl9jlymj4kj7mrt7n0s7yt44p3fd66/T/tmpkJ9wvz', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train

({'accuracy': 0.96, 'global_step': 165, 'loss': 0.15013154}, [])